In [1]:
import os
import requests
import json
import glob
import rasterio.mask
import fiona

import numpy as np
import rasterio as rio
import matplotlib.pyplot as plt
import geopandas as gpd
from rasterio.plot import show
from rasterio.plot import show_hist

In [2]:
with open('api_key.txt', 'r') as f:
    token = f.read().strip()

In [3]:
dataset_id = 'ea71ef6e-9569-49fc-be16-ba98d876fb73'

scenes = [
    '12d8698a-4553-4540-9f31-076df70e9616'
]

In [4]:
# データセットIDとシーンIDから、そのシーンに含まれるファイル名を取得
def get_dataset_data_by_id_files(base_path, token, dataset_id, data_id):
    print(base_path, token, dataset_id, data_id)
    url = f'{base_path}/api/traveler/v1/datasets/{dataset_id}/data/{data_id}/files/'
    headers = {
        'Authorization': f'Bearer {token}',
        'content-type': 'application/json'
    }
    
    r = requests.get(url, headers=headers)
    print(r.status_code)
    assert r.status_code == 200
    return json.loads(r.content)

In [5]:
# ファイルIDを指定して、ダウンロードURLを取得
def get_dataset_data_by_id_files_by_id_download_url(base_path, token, dataset_id, data_id, file_id):
    url = f'{base_path}/api/traveler/v1/datasets/{dataset_id}/data/{data_id}/files/{file_id}/download-url/'
    headers = {
        'Authorization' : f'Bearer {token}',
        'content-type': 'application/json'
    }
    r = requests.post(url, headers=headers)
    assert r.status_code == 200
    return json.loads(r.content)

In [6]:
def download(scenes, dataset_id, token, dist='./', base_path='https://www.tellusxdp.com'):
    for scene_id in scenes:
        files = get_dataset_data_by_id_files(base_path, token, dataset_id, scene_id)
        print(files)
        rawdata = files['results']
        path = os.path.join(dist, scene_id)
        if len(rawdata) > 0:
            for file in rawdata:
                file_id = file['id']
                file_name = file['name']
                
                file_path = os.path.join(path, file_name)
                
                download_url = get_dataset_data_by_id_files_by_id_download_url(base_path, token, dataset_id, scene_id, file_id)['download_url']
                
                r = requests.get(download_url, stream=True)
                
                if not os.path.exists(path):
                    os.makedirs(path)
                
                with open(file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                            f.flush()

In [7]:
download(scenes, dataset_id, token, dist='../data')

https://www.tellusxdp.com QU2R@jPB@DTyianyivD97kvvYKG6bnpY ea71ef6e-9569-49fc-be16-ba98d876fb73 12d8698a-4553-4540-9f31-076df70e9616
200
{'results': [{'size_bytes': 83374760, 'mime_type': 'image/tiff', 'name': 'IMG-03-ALAV2A278862760-OORIRFU_000.tif', 'id': 1, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 374558, 'mime_type': 'image/png', 'name': 'ALAV2A278862760-OORIRFU-D560P3-20110419-000_thumb.png', 'id': 2, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 335890, 'mime_type': 'image/png', 'name': 'ALAV2A278862760-OORIRFU_000.png', 'id': 3, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 83374760, 'mime_type': 'image/tiff', 'name': 'IMG-01-ALAV2A278862760-OORIRFU_000.tif', 'id': 4, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 978, 'mime_type': 'application/xm